In [75]:
import pandas as pd
def to_dataframe(texts):
    ids = []
    labels = []
    parents = []
    info = []
    levels = []
    label_length = []

    for idx, text in enumerate(texts):
        parts = text.split('-')
        for i, part in enumerate(parts):
            ids.append("-".join(parts[:i+1]))
            labels.append(part)
            parents.append("-".join(parts[:i]) if i > 0 else "")
            info.append(part)
            levels.append(i+1)
            label_length.append(len(part))

            
            
    return pd.DataFrame({
        "ids": ids,
        "labels": labels,
        "parents": parents,
        "info": info,
        "level": levels,
        "label_length": label_length,
    })
    
# 函数功能：将文本转换为dataframe


In [101]:
with open('example.txt', 'r') as f:
    example = f.read()

input_list = example.split('\n')
# print(input_list)
input_list = to_dataframe(input_list)
# print(input_list)

In [77]:
# 打印df的标题
print(input_list.columns)

Index(['ids', 'labels', 'parents', 'info', 'level', 'label_length'], dtype='object')


In [78]:
# 答应df的标题
print(input_list.columns)
# 打印df.length
print(input_list.label_length)

Index(['ids', 'labels', 'parents', 'info', 'level', 'label_length'], dtype='object')
0       8
1      40
2     321
3       7
4      41
5     403
6      10
7      38
8     392
9       0
10      0
11      0
Name: label_length, dtype: int64


In [79]:
import textwrap
def auto_wrap_text(text, max_width):
    wrapped_text = textwrap.wrap(text, max_width)
    wrapped_text = "<br>".join(wrapped_text)
    return wrapped_text

def apply_wrap(row):
    if row['level'] == 1:
        return auto_wrap_text(row['labels'], max_width=10)
    elif row['level'] == 2:
        return auto_wrap_text(row['labels'], max_width=20)
    else:
        return auto_wrap_text(row['labels'], max_width=20)


# 尝试不同的内容的换行
# 不同level的换行

In [102]:
import random
import colorsys

base_hue = random.random()  # 在Python的colorsys模块中，色调的范围是0-1，所以我们需要把角度除以360

# 生成色系相似的8个颜色
base_colors = []
for i in range(8):
    # 在色调上加上一个小的偏移（在这里是i*10度），并确保结果在0-1范围内
    hue = (base_hue + i*10/360) % 1
    # 使用饱和度1和亮度0.5生成颜色（这将生成一个鲜艳的颜色）
    color = colorsys.hls_to_rgb(hue, 0.5, 1)
    # 将颜色从RGB空间转换到十六进制颜色代码
    base_colors.append("#"+''.join([f"{int(val*255):02x}" for val in color]))
    

# 转换十六进制颜色代码为RGB颜色值（范围是0-1）
base_colors_rgb = [[int(color[i:i+2], 16) / 255 for i in range(1, 6, 2)] for color in base_colors]

# 节点和层级的关系，假设我们有8个段落，每个段落有3个层级
nodes = ["P{}-L{}".format(p, l) for p in range(8) for l in range(3)]
levels = [l for p in range(8) for l in range(3)]

# 根据节点的层级和对应的基础颜色来生成颜色
node_colors = []
for i in range(8):
    for j in range(3):
        # 随着层级增加，将颜色的亮度增加
        h, l, s = colorsys.rgb_to_hls(*base_colors_rgb[i])
        color = colorsys.hls_to_rgb(h, min(1, l + 0.2*j), s)  # 每次增加20%的亮度，最大为1
        node_colors.append("rgba"+str(tuple([int(255*val) for val in color] + [1.0])) + ")")

In [158]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import pandas as pd
import json
import textwrap
import numpy as np
import plotly.colors as colors
import matplotlib.pyplot as plt
import random
import colorsys



# 读取文本
with open('example.txt', 'r') as f:
    example = f.read()
input_list = example.split('\n')
input_list = to_dataframe(input_list)


# 生成模拟数据
df = pd.DataFrame(input_list)
df["labels"] = df.apply(apply_wrap, axis=1)
# 计算中间值

midpoint = np.average(df['label_length'])

# 将 'label_length' 列的数据标准化到 0 到 1 的范围
df['color_scale'] = (df['label_length'] - df['label_length'].min()) / (df['label_length'].max() - df['label_length'].min())


# 将标准化后的 'color_scale' 列的数据再次标准化到 -0.5 到 0.5 的范围
df['color_scale'] = (df['color_scale'] - 0.5)* 0.5


fig = go.Figure(go.Treemap(
    ids=df['ids'],
    labels=df['labels'],
    parents=df['parents'],

    maxdepth=2,
    # text=df['labels'],
    insidetextfont=dict(size=30, color='#FFFFFF'),
    outsidetextfont=dict(size=15, color='#FF0000', family='Arial'),


    marker=dict(

        colors=generate_linear_list(len(df['color_scale'])),
        # colors=df['color_scale'],  # 使用label_length列的值来定义颜色
        colorscale=[(0, 'rgb(242, 205, 172)'), (1, 'rgb(89, 89, 89)')],  # 浅红色到浅蓝色的自定义颜色比例尺
        reversescale=True,  # 颜色比例尺反转，从红色到蓝色
    ),
    # marker_colors=node_colors,  # 使用标签长度作为权重
    
    tiling=dict(
    packing='squarify', # slice-dice  # Set packing algorithm to 'squarify' # 布局方式
    pad=0.2
    )
))

fig.update_layout(
    # title="人工智能在不同行业的应用",  # 添加标题
    title={
    'text': "<b>人工智能在不同行业的应用",
    'x': 0.5,  # 将标题的 x 位置设置为 0.5，即图形的中心
    'xanchor': 'center',  # 将标题的 x 锚点设置为 'center'，即标题的中心
    # 'y' : 0.96 , # 高度调整
    },
    title_font=dict(
    family="Courier New, monospace",  # 设置字体
    size=20,  # 设置字体大小
    color="RebeccaPurple"  # 设置字体颜色
    ),
    # uniformtext=dict(minsize=15, mode='hide'), 
    margin = dict(t=50, l=25, r=25, b=25),
    width=1200,
    height=900,
    plot_bgcolor='rgba(0, 0, 0, 0)' ,

)


fig.update_traces(marker=dict(cornerradius=20)) # 圆角
fig.show()




In [129]:
import random

def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')  # remove the '#'
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def generate_color_list(hex_color_list, length):
    return [hex_to_rgb(random.choice(hex_color_list)) for _ in range(length)]

# 使用示例
hex_color_list = ["#232625", "#35403A", "#4C594F", "#A4A69C","#BFBFB8"]
print(generate_color_list(hex_color_list, 10))


[(164, 166, 156), (35, 38, 37), (164, 166, 156), (76, 89, 79), (76, 89, 79), (76, 89, 79), (53, 64, 58), (191, 191, 184), (53, 64, 58), (191, 191, 184)]


In [157]:
hex_to_rgb("#F2CDAC"),hex_to_rgb("#595959")

((242, 205, 172), (89, 89, 89))

In [29]:
import plotly.express as px
import numpy as np
df = px.data.gapminder().query("year == 2007")
fig = px.treemap(df, path=[px.Constant("world"), 'continent', 'country'], values='pop',
                  color='lifeExp', hover_data=['iso_alpha'],
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(df['lifeExp'], weights=df['pop']))
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [72]:
import plotly.graph_objects as go
import pandas as pd

# 创建一个数据帧，其中包含节点的标签、父节点和颜色值
df = pd.DataFrame({
    'labels': ["Node1", "Node2", "Node3", "Node4", "Node5"],
    'parents': ["", "Node1", "Node1", "Node2", "Node3"],
    'values': [0, 1, 1, 1, 1]
})

# 找出根节点，这里假设根节点就是那些父节点为空的节点
root_node = df[df['parents'] == ""]
other_nodes = df[df['parents'] != ""]

fig = go.Figure()

# 为根节点创建一个单独的 Treemap trace
fig.add_trace(go.Treemap(
    labels=root_node['labels'],
    parents=root_node['parents'],
    marker=dict(
        colors=['green'],  # 设定根节点的颜色为绿色
        showscale=False
    ),
    name=""
))

# 为其它节点创建一个单独的 Treemap trace
fig.add_trace(go.Treemap(
    labels=other_nodes['labels'],
    parents=other_nodes['parents'],
    marker=dict(
        colors=other_nodes['values'],  # 使用 'values' 列的值设定节点的颜色
        colorscale='Reds',  # 设定颜色比例尺为 'Reds'
        showscale=False
    ),
    name=""
))

fig.show()


In [73]:
import plotly.graph_objects as go

# 创建一个数据帧，其中包含节点的标签、父节点和颜色值
df = pd.DataFrame({
    'labels': ["Root", "A", "B", "C", "D", "E"],
    'parents': ["", "Root", "Root", "A", "B", "B"],
    'values': [0, 1, 1, 1, 1, 1]
})

fig = go.Figure()

# 为根节点创建一个单独的 Treemap trace
fig.add_trace(go.Treemap(
    labels=["Root"],
    parents=[""],
    marker=dict(
        colors=['rgba(0, 0, 0, 0)'],  # 设定根节点的颜色为透明
        showscale=False
    ),
    name=""
))

# 为其它节点创建一个单独的 Treemap trace
fig.add_trace(go.Treemap(
    labels=df['labels'][1:],
    parents=df['parents'][1:],
    marker=dict(
        colors=df['values'][1:],  # 使用 'values' 列的值设定节点的颜色
        colorscale='Reds',  # 设定颜色比例尺为 'Reds'
        showscale=False
    ),
    name=""
))

fig.show()


In [4]:
import pandas as pd
import re
def markdown_to_df(md_lines):
    # 初始化存储ids、labels、parents和levels的空列表
    ids = []
    labels = []
    parents = []
    levels = []
    info = []
    # 初始化父级列表，存储每一级的父级标题
    parent_list = [""] * 10



    for line in md_lines:
        # 这里假设输入是一个行的列表，而不是一个文件
        stripped_line = line.lstrip(" ")
        # 如果行以“数字.**”开头，就认为这是一个一级标题
        if re.match(r"\d+\.\s\*\*", stripped_line):
            level = 1
        else:
            level = line.count(" ", 0, len(line) - len(stripped_line)) // 2 + 1
        label = stripped_line[level:].strip('.').strip().strip('*')
        parent = parent_list[level-1]
        ids.append(label if parent=="" else parent + '/' + label)
        labels.append(label)
        parents.append("" if parent=="" else parent if level==1 else parent_list[level-2]+'/'+parent)
        levels.append(level)
        parent_list[level] = label
        # 给info列提供一些默认值，你可以替换为实际的数据
        info.append(label)


    df = pd.DataFrame({
        "ids": ids,
        "labels": labels,
        "parents": parents,
        "info": info,
        "level": levels
    })

    return df

import random

def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')  # remove the '#'
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def generate_color_list(hex_color_list, length):
    return [hex_to_rgb(random.choice(hex_color_list)) for _ in range(length)]

# 使用示例
hex_color_list = ["#232625", "#35403A", "#4C594F", "#A4A69C","#BFBFB8"]
print(generate_color_list(hex_color_list, 10))




def read_markdown_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        # 使用列表推导式去掉每行的换行符
        return [line.rstrip('\n') for line in file.readlines()]

md_lines = read_markdown_file('test1.md')

df = markdown_to_df(md_lines)
print(df)




                             ids             labels  \
0                   人工智能的历史与理论基础       人工智能的历史与理论基础   
1      人工智能的历史与理论基础/早期的人工智能研究与应用       早期的人工智能研究与应用   
2            早期的人工智能研究与应用/器人初期研究             器人初期研究   
3              器人初期研究/iza和Shakey         iza和Shakey   
4           iza和Shakey/za的对话系统原理          za的对话系统原理   
..                           ...                ...   
130              数据隐私和伦理考量/护数据隐私              护数据隐私   
131             数据隐私和伦理考量/I的伦理问题             I的伦理问题   
132  人工智能的未来发展/人工智能的影响：工作、社会、经济等  人工智能的影响：工作、社会、经济等   
133    人工智能的影响：工作、社会、经济等/I对工作的影响            I对工作的影响   
134    人工智能的影响：工作、社会、经济等/I对社会的影响            I对社会的影响   

                         parents               info  level  
0                                      人工智能的历史与理论基础      1  
1                  /人工智能的历史与理论基础       早期的人工智能研究与应用      2  
2      人工智能的历史与理论基础/早期的人工智能研究与应用             器人初期研究      3  
3            早期的人工智能研究与应用/器人初期研究         iza和Shakey      4  
4              器人初期研究/iza和Shakey  

In [5]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import pandas as pd
import json
import textwrap
import numpy as np
import plotly.colors as colors
import matplotlib.pyplot as plt
import random
import colorsys



# # 读取文本
# with open('example.txt', 'r') as f:
#     example = f.read()
# input_list = example.split('\n')
# input_list = to_dataframe(input_list)


# # 生成模拟数据
# df = pd.DataFrame(input_list)
# df["labels"] = df.apply(apply_wrap, axis=1)
# # 计算中间值

midpoint = np.average(df['label_length'])

# 将 'label_length' 列的数据标准化到 0 到 1 的范围
df['color_scale'] = (df['label_length'] - df['label_length'].min()) / (df['label_length'].max() - df['label_length'].min())


# 将标准化后的 'color_scale' 列的数据再次标准化到 -0.5 到 0.5 的范围
df['color_scale'] = (df['color_scale'] - 0.5)* 0.5


fig = go.Figure(go.Treemap(
    ids=df['ids'],
    labels=df['labels'],
    parents=df['parents'],

    maxdepth=2,
    # text=df['labels'],
    insidetextfont=dict(size=30, color='#FFFFFF'),
    outsidetextfont=dict(size=15, color='#FF0000', family='Arial'),


    marker=dict(

        colors=generate_linear_list(len(df['color_scale'])),
        # colors=df['color_scale'],  # 使用label_length列的值来定义颜色
        colorscale=[(0, 'rgb(242, 205, 172)'), (1, 'rgb(89, 89, 89)')],  # 浅红色到浅蓝色的自定义颜色比例尺
        reversescale=True,  # 颜色比例尺反转，从红色到蓝色
    ),
    # marker_colors=node_colors,  # 使用标签长度作为权重
    
    tiling=dict(
    packing='squarify', # slice-dice  # Set packing algorithm to 'squarify' # 布局方式
    pad=0.2
    )
))

fig.update_layout(
    # title="人工智能在不同行业的应用",  # 添加标题
    title={
    'text': "<b>人工智能在不同行业的应用",
    'x': 0.5,  # 将标题的 x 位置设置为 0.5，即图形的中心
    'xanchor': 'center',  # 将标题的 x 锚点设置为 'center'，即标题的中心
    # 'y' : 0.96 , # 高度调整
    },
    title_font=dict(
    family="Courier New, monospace",  # 设置字体
    size=20,  # 设置字体大小
    color="RebeccaPurple"  # 设置字体颜色
    ),
    # uniformtext=dict(minsize=15, mode='hide'), 
    margin = dict(t=50, l=25, r=25, b=25),
    width=1200,
    height=900,
    plot_bgcolor='rgba(0, 0, 0, 0)' ,

)


fig.update_traces(marker=dict(cornerradius=20)) # 圆角
fig.show()




KeyError: 'label_length'